In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
%matplotlib inline
# NLP packages
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords

# Modeling packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# faculty_reviews = pd.read_csv('/content/clgstuff/rpm.csv')
faculty_reviews = pd.read_csv("https://0x0.st/HWh-.csv")

faculty_reviews.rename(columns = {'tid':'id','comment':'review','quality':'rating'}, inplace = True)
print(faculty_reviews.head(3))

faculty_reviews.rating.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=["azure", "lightgrey", "thistle","khaki","pink"])
percent_val = 100 * faculty_reviews['rating'].value_counts()/len(faculty_reviews)
percent_val
faculty_reviews.loc[faculty_reviews.rating == "average", "Sentiment_rating"] = "netural"
faculty_reviews.to_csv("dasdata.csv",index=False)
file1 = open("dasdata.csv", "a")  # append mode
# file1.write('111,"nor good not best",netural')
# file1.write('\n111,"Average faculty ",netural')
# file1.write('\n111,"Average",netural')

file1.close()

In [ ]:
faculty_reviews.drop(faculty_reviews.index[faculty_reviews.rating == "good"], inplace=True)
faculty_reviews.drop(faculty_reviews.index[faculty_reviews.rating == "poor"], inplace=True)
# faculty_reviews.loc(faculty_reviews.index[faculty_reviews.rating == "good"], inplace=True)
# faculty_reviews.loc[faculty_reviews.rating == "good", "Sentiment_rating"] = "positive"
faculty_reviews.loc[faculty_reviews.rating == "awesome", "Sentiment_rating"] = "positive"
# faculty_reviews.loc[faculty_reviews.rating == "poor", "Sentiment_rating"] = "netural"
faculty_reviews.loc[faculty_reviews.rating == "awful", "Sentiment_rating"] = "negative"
faculty_reviews.loc[faculty_reviews.rating == "average", "Sentiment_rating"] = "netural"

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
faculty_reviews['reviews_text_new'] = faculty_reviews['review'].str.lower()
spl_chars = faculty_reviews['reviews_text_new'].apply(lambda review:
                                                     [char for char in list(review) if not char.isalnum() and char != ' '])

## Getting list of list into a single list
flat_list = [item for sublist in spl_chars for item in sublist]

review_backup = faculty_reviews['reviews_text_new'].copy()
faculty_reviews['reviews_text_new'] = faculty_reviews['reviews_text_new'].str.replace(r'[^A-Za-z0-9 ]+', ' ')
stop_words = stopwords.words('english')
def stopwords_removal(stop_words, sentence):
    return [word for word in nltk.word_tokenize(sentence) if word not in stop_words]

faculty_reviews['reviews_text_nonstop'] = faculty_reviews['reviews_text_new'].apply(lambda row: stopwords_removal(stop_words, row))
faculty_reviews[['reviews_text_new','reviews_text_nonstop']]